In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d uciml/breast-cancer-wisconsin-data
!unzip \*.zip && rm *.zip

  0% 0.00/48.6k [00:00<?, ?B/s]
100% 48.6k/48.6k [00:00<00:00, 1.82MB/s]
Archive:  breast-cancer-wisconsin-data.zip
  inflating: data.csv                


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import skfuzzy as fuzz

# Step 1: Load Data
data = pd.read_csv('/content/data.csv')

# Assuming 'diagnosis' is the target variable
X = data[['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
          'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean']]
y = data['diagnosis']  # Target variable

# Step 2: Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Fuzzy Memberships Calculation
m = 2  # Fuzziness coefficient
k = 5  # Number of neighbors
knn_model = KNeighborsClassifier(n_neighbors=k)
knn_model.fit(X_train_scaled, y_train)
distances, indices = knn_model.kneighbors(X_train_scaled)
neighbor_labels = y_train.iloc[indices]
n_classes = len(np.unique(y_train))
membership_matrix = np.zeros((len(X_train_scaled), n_classes))
for i in range(len(X_train_scaled)):
    for j in range(k):
        neighbor_class = neighbor_labels.iloc[i, j]
        membership_matrix[i, neighbor_class] += 1 / distances[i, j] ** (2 / (m - 1))
membership_matrix = membership_matrix / np.sum(membership_matrix, axis=1).reshape(-1, 1)

# Step 5: Model Training - FKNN
def predict_fuzzy_membership(x):
    distances, indices = knn_model.kneighbors([x])
    neighbor_labels = y_train.iloc[indices[0]]
    fuzzy_membership = np.zeros(n_classes)
    for j in range(k):
        neighbor_class = neighbor_labels.iloc[j]
        fuzzy_membership[neighbor_class] += 1 / distances[0, j] ** (2 / (m - 1))
    return fuzzy_membership / np.sum(fuzzy_membership)

y_pred = []
for x_test_point in X_test_scaled:
    fuzzy_membership = predict_fuzzy_membership(x_test_point)
    predicted_class = np.argmax(fuzzy_membership)
    y_pred.append(predicted_class)

# Step 6: Model Evaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

print("Classification Report:\n", classification_report(y_test, y_pred))


ModuleNotFoundError: No module named 'skfuzzy'